# ENGR-E 533 Deep Learning Systems - Assignment 3

## Name : Akhila Sakiramolla (asakiram@iu.edu)
## UID : 2000886005

## Importing required libraries

In [24]:
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import IPython.display as ipd
import cv2
import os
import soundfile as sf
import tensorflow as tf
import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import initializers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.callbacks import History 
from keras.layers.core import Dense, Activation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM

# To execute a cell line by line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Problem 3: Speech Denoising Using RNN

In [25]:
# Getting the path for all files

path = '/content/drive/MyDrive/Colab Notebooks/DLS/timit-homework'

path_tr = path+'/tr/'
path_te = path+'/te/'
path_v = path+'/v/'

In [26]:
# Function to load files from the path

def dataLoad(path, file_name):
  data = []
  for file in os.listdir(path):
    if file.startswith(file_name):
      data.append(file)
  data = sorted(data)
  return data

In [27]:
# Loading all train files
trn = dataLoad(path_tr, 'trn')
trs = dataLoad(path_tr, 'trs')
trx = dataLoad(path_tr, 'trx')

# Loading all validation files
vn = dataLoad(path_v, 'vn')
vs = dataLoad(path_v, 'vs')
vx = dataLoad(path_v, 'vx')

# Loading all test files
tex = dataLoad(path_te, 'tex')

In [28]:
# Function to convert files into spectograms

def stft(path, file):
  spect = []
  spect_mag = []
  for i in file:
    file_name = path+i
    s, sr = librosa.load(file_name, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    spect.append(S)
    s_abs = np.abs(S)
    spect_mag.append(s_abs)
  
  return spect, spect_mag, sr

In [29]:
# Train files
n_tr_spect, n_tr_mag, sr = stft(path_tr, trn)
s_tr_spect, s_tr_mag, sr = stft(path_tr, trs)
x_tr_spect, x_tr_mag, sr = stft(path_tr, trx)

# Validation files
n_v_spect, n_v_mag, sr = stft(path_v, vn)
s_v_spect, s_v_mag, sr = stft(path_v, vs)
x_v_spect, x_v_mag, sr = stft(path_v, vx)

In [30]:
# Function to pad the data

def padData(mat):
  for i in range(len(mat)):
    dim = mat[i].shape
    padding = 10 - (dim[1] % 10)
    alpha = np.random.normal(0,0.00001, size = [dim[0],int(padding)])
    mat[i] = np.concatenate((mat[i],alpha), axis = 1)

In [31]:
# Train files
padData(n_tr_mag)
padData(s_tr_mag)
padData(x_tr_mag)

# Validation files
padData(n_v_mag)
padData(s_v_mag)
padData(x_v_mag)

In [32]:
# Creating IBM matrix

mask_tr = []
for s, n in zip(s_tr_mag,n_tr_mag):
  mask_tr.append((s>n)*1)

mask_v = []
for s, n in zip(s_v_mag,n_v_mag):
  mask_v.append((s>n)*1)

In [33]:
# Creating training bacthes for model

train_batch = []
for i in x_tr_mag:
  batch = []
  for j in range(i.shape[1]//10):
    batch1 = i[:,10*j:10*j+10]
    batch.append(batch1)
  batch = np.array(batch)
  train_batch.append(batch)
train_batch = np.concatenate(train_batch)

mask_tr_batch = []
for i in mask_tr:
  batch = []
  for j in range(i.shape[1]//10):
    batch1 = i[:,10*j:10*j+10]
    batch.append(batch1)
  batch = np.array(batch)
  mask_tr_batch.append(batch)
mask_tr_batch = np.concatenate(mask_tr_batch)

In [34]:
# Creating validation bacthes for model

val_batch = []
for i in x_v_mag:
  batch = []
  for j in range(i.shape[1]//10):
    batch1 = i[:,10*j:10*j+10]
    batch.append(batch1)
  batch = np.array(batch)
  val_batch.append(batch)
val_batch = np.concatenate(val_batch)

mask_v_batch = []
for i in mask_v:
  batch = []
  for j in range(i.shape[1]//10):
    batch1 = i[:,10*j:10*j+10]
    batch.append(batch1)
  batch = np.array(batch)
  mask_v_batch.append(batch)
mask_v_batch = np.concatenate(mask_v_batch)

In [35]:
# Building an LSTM model

model = Sequential()
model.add(Input(shape=(None,513)))
model.add(LSTM(activation='relu',units=512, return_sequences=True, kernel_initializer='he_normal'))
model.add(LSTM(activation='relu',units=512, return_sequences=True, kernel_initializer='he_normal'))
model.add(Dense(units = 513, activation='sigmoid', kernel_initializer = tf.keras.initializers.HeNormal()))

model.compile(loss='binary_crossentropy',optimizer='adam')

In [36]:
# Train datasets
X_train = np.transpose(train_batch,(0,2,1))
Y_train = np.transpose(mask_tr_batch,(0,2,1))

# Validation datasets
X_val =  np.transpose(val_batch,(0,2,1))
Y_val = np.transpose(mask_v_batch,(0,2,1))

In [37]:
# Fitting the model

hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = 100, batch_size = 512)

Epoch 1/100
25/25 [==============================] - 5s 100ms/step - loss: 0.6613 - val_loss: 0.6260
Epoch 2/100
25/25 [==============================] - 1s 56ms/step - loss: 0.6009 - val_loss: 0.5670
Epoch 3/100
25/25 [==============================] - 1s 57ms/step - loss: 0.5603 - val_loss: 0.5339
Epoch 4/100
25/25 [==============================] - 1s 56ms/step - loss: 0.5326 - val_loss: 0.5124
Epoch 5/100
25/25 [==============================] - 1s 57ms/step - loss: 0.5131 - val_loss: 0.5016
Epoch 6/100
25/25 [==============================] - 1s 56ms/step - loss: 0.5028 - val_loss: 0.4897
Epoch 7/100
25/25 [==============================] - 1s 56ms/step - loss: 0.4878 - val_loss: 0.4749
Epoch 8/100
25/25 [==============================] - 1s 58ms/step - loss: 0.4776 - val_loss: 0.4743
Epoch 9/100
25/25 [==============================] - 1s 57ms/step - loss: 0.4682 - val_loss: 0.4656
Epoch 10/100
25/25 [==============================] - 1s 56ms/step - loss: 0.4681 - val_loss: 0.462

In [38]:
# Predicting  on validation data

vx1 = [i  for i in vx if "(1)" not in i]
x_v_spect, x_v_mag, sr = stft(path_v, vx1)
model_preds = []
for i in range(len(x_v_spect)):
  spect = x_v_spect[i]
  mag = x_v_mag[i]
  mag1 = mag.T.reshape([1,-1,513])
  pred = model.predict(mag1)
  pred = np.round(pred.squeeze())
  pred1 = (pred.T)*spect
  model_preds.append(pred1)

1/1 [==============================] - 0s 42ms/step


In [39]:
# Calculating SNR value

snr_vals = np.zeros(len(s_v_spect))
for i in range(len(s_v_spect)):
  s1 = librosa.istft(s_v_spect[i], hop_length=512)
  s2 = librosa.istft(model_preds[i], hop_length=512)
  test = s1[:len(s2)]
  pred = s2[:len(s1)]
  snr = 10*np.log10(np.divide(np.square(test).sum(), np.square(test - pred).sum()))
  snr_vals[i] = snr

snr_final = np.mean(snr_vals)
print("The SNR value is : ", snr_final)

The SNR value is :  10.462351281444231


In [40]:
# Predciting on test data

x_te_spect, x_te_mag, sr = stft(path_te, tex)
model_preds1 = []
for i in range(len(x_te_spect)):
  spect = x_te_spect[i]
  mag = x_te_mag[i]
  mag1 = mag.T.reshape([1,-1,513])
  pred = model.predict(mag1)
  pred = np.round(pred.squeeze())
  pred1 = (pred.T)*spect
  s = librosa.istft(pred1, hop_length=512)
  model_preds.append(pred1)

  file = str(i)
  file_name = 'tec' + '0000'[:-len(file)]+file
  sf.write(f'/content/drive/MyDrive/Colab Notebooks/DLS/timit-homework/te_final/{file_name}.wav', s, sr)

1/1 [==============================] - 0s 46ms/step


In [41]:
# Checking for a sample for original and predicted 

orig, sr = librosa.load("/content/drive/MyDrive/Colab Notebooks/DLS/timit-homework/te/tex0000.wav", sr=None)
ipd.Audio(orig, rate = sr)

pred, sr = librosa.load("/content/drive/MyDrive/Colab Notebooks/DLS/timit-homework/te_final/tec0000.wav", sr=None)
ipd.Audio(pred, rate = sr)